---
title: Obtuse Units
execute:
    echo: true
website:
    page-navigation:
---




Tired of using simple, pedestrian units like "feet" or "hours"? Do you yearn to be technically accurate yet utterly intractable? Want to speak in a way that says "Wow, now *this* guy is a tool"? Just use this handy generator to convert any statement into a pointlessly convoluted (yet correct) combination of SI units.

This project used to be a Twitter bot, back when Twitter was just incidentally the worst thing to happen to human communication and not intentionally the worst thing to happen to human communication. Mercifully, I put the bot on sleep the moment it was born and tested; now you can use the program for yourself.


<!---   Imports    --->


In [ ]:
#Builtin
import numpy as np
import random
import re
import time
#Extenals
from quantulum3 import parser
from pint import UnitRegistry

<!---   User Input  --->

<!---   Config  --->


In [ ]:
#List of SI derived unit to be used.
#(Unit name, Unit dimesions [M,L,T,A], Unit count [0 for no bias])
derived_units = np.array([('hertz',   ( 0, 0,-1, 0) ,0),
                          ('newton',  ( 1, 1,-2, 0) ,0),
                          ('pascal',  ( 1,-1,-2, 0) ,0),
                          ('joule',   ( 1, 2,-2, 0) ,0),
                          ('watt',    ( 1, 2,-3, 0) ,0),
                          ('coulomb', ( 0, 0, 1, 1) ,0),
                          ('volt',    ( 1, 2,-3,-1) ,0),
                          ('farad',   (-1,-2, 4, 2) ,0),
                          ('ohm',     ( 1, 2,-3,-2), 0),
                          ('siemen',  (-1,-2, 3, 2), 0),
                          ('weber',   ( 1, 2,-2,-1), 0),
                          ('tesla',   ( 1, 0,-2,-1), 0),
                          ('henry',   ( 1, 2,-2,-2), 0),
                          ('sievert', ( 0, 2,-2, 0), 0)],
                          dtype=[('uname','U10'), ('dims',np.ndarray),('count',np.float32)])

#List of SI prefixes and their associated powers
#Removing prefixes in the range (-1,1) may cause basis spanning issues in calls with very tight user tolerances.
prefixes = np.array([('yotta', 24),
                     ('zetta', 21),
                     ('exa',   18),
                     ('peta',  15),
                     ('tera',  12),
                     ('giga',   9),
                     ('mega',   6),
                     ('kilo',   3),
                     ('hecto',  2),
                     ('deca',   1),
                     ('',       0),
                     ('deci',  -1),
                     ('centi', -2),
                     ('milli', -3),
                     ('micro', -6),
                     ('nano',  -9),
                     ('pico', -12),
                     ('femto',-15),
                     ('atto', -18),
                     ('zepto',-21),
                     ('yocto',-24)],
                     dtype=[('name','U10'), ('mag',np.int8)])

<!---   Core conversion function    --->


In [ ]:
def obtusify(val, idims, SI, pref, loops = 2, maxvalord = None, minvalord = None, maxprefs = None, spread = None):
    '''
    Given a quantity in SI base units, returns an equivalent value in an obtuse combination of
      derived SI units and prefixes

    Args:
      val   -- float : numerical value of quantity in base units
      idims -- (4,) : tuple corresponding to dimensionality of input quantity (M,L,T,I)
      SI    -- (n,5) struct ndarray : information on SI derived units, see config
      pref  -- (n,2) struct ndarray : indicating a set of SI prefixes to superfluously add

    Kwargs:
      loops       -- int : Number of derived units to be present in the obfuscation. Set it irresponsibly high!
      maxvalord   -- int : If not None, maximum order of output numeric value
      minvalord   -- int : If not None, minimum order of output numeric value
      maxprefs    -- int : If not None, maximum number of SI prefixes to include in output
      spread      -- float [0,1) : If not None, creates a bias towards including more individual unit names
                       in the output (vs. less unique names with higher powers). Strong interaction with loops.

    Returns:
      ostring -- string : quantity equal to input, expressed obtusely

    Sample Usage:
        # Obtusification of 8024 metres
        obtusify(8024, (0,1,0,0), config.SI, config.pref) --> "8.024 microsievert-kilograms per nanonewton"
    '''

    #Basic input checks
    if maxvalord == None: maxvalord =  np.inf
    if minvalord == None: minvalord = -np.inf
    if maxvalord - minvalord < 0:
        print("You set the output value order minimum higher than the maximum. Don't think I don't see you trying to crash the prefix algorithm.")
        raise Exception

    #Pick a random non-zero positive dimension
    ind = random.choice([i for i,x in enumerate(idims) if x > 0])

    #Pick a random SI derived unit which is nonzero positive in this dimension (ensures at least one unit on numerator)
    SIind = random.choice([i for i,x in enumerate(SI['dims']) if x[ind] > 0])
    SI['count'][SIind] += 1

    #Check which dimensions remain to be filled
    wSI = [np.array(x)*y for x,y in zip(SI['dims'], SI['count'])]
    remainder = idims - np.sum(wSI, axis=0)


    #Loop while dimensions are not matched
    i = 0
    while i < loops:
        i += 1

        #Find the derived unit which would most decrease the remainder if added to the numerator
        diffs = np.sum(np.abs([list(x) - remainder for x in SI['dims']]), axis=1).astype(np.float32)
        diffs[SI['count'] < 0] = np.nan #Ignore options which would simply cancel out existing units
        n,ndiff = np.where(diffs == np.nanmin(diffs))[0][0], np.nanmin(diffs)
        #Find the derived unit which would most decrease the remainder if added to the denominator
        diffs = np.sum(np.abs([list(x) + remainder for x in SI['dims']]), axis=1).astype(np.float32)
        diffs[SI['count'] > 0] = np.nan
        d,ddiff = np.where(diffs == np.nanmin(diffs))[0][0], np.nanmin(diffs) #when added to the denominator

        #Sometimes selecting the unit which best moves the dimensionality towards the target actually produces
        #  lackluster results, e.g. feedback loops of two units undoing eachother around the target.
        #These options are ways to introduce bias for more interesting results
        if spread is not None and loops - i > 3:
            #Adds a chance to instead select a random unused unit if possible
            if np.random.rand() > spread and 0 in SI['count']:
                uu = random.choice([i for i,x in enumerate(SI['count']) if x == 0])
                if np.random.rand() > .5:
                    SI['count'][uu] += 1 #Slap it on the numerator
                else:
                    SI['count'][uu] -= 1 #Slap it on the denominator
        else:
            if ndiff <= ddiff:
                SI['count'][n] += 1
            else:
                SI['count'][d] -= 1

        #Check which dimensions remain to be filled
        wSI = [np.array(x)*y for x,y in zip(SI['dims'], SI['count'])]
        remainder = idims - np.sum(wSI, axis=0)

    #Assign SI prefixes
    nslots = np.sum(SI['count'] > 0) + np.sum(remainder > 0) #Number of slots available on numerator
    dslots = np.sum(SI['count'] < 0) + np.sum(remainder < 0) #Number of slots available on denominator
    pn = np.full(nslots, None)
    pd = np.full(dslots, None)

    #Bound the number of prefixes to be added
    if maxprefs is not None:
        nempty, dempty = np.sum(pn == None), np.sum(pd == None)
        while nempty + dempty > maxprefs:
            #Fill a random slot with an empty prefix (uniform selection across all slots)
            ratio = nempty/(nempty + dempty)
            if np.random.rand() < ratio:
                i = [i for i,x in enumerate(pn) if x == None]
                pn[random.choice(i)] = 0
            else:
                i = [i for i,x in enumerate(pd) if x == None]
                pd[random.choice(i)] = 0
            nempty, dempty = np.sum(pn == None), np.sum(pd == None)

    #Assign arbitrary prefixes to open slots slots, avoiding dupes
    for i in [i for i,x in enumerate(pn) if x == None]:
        pn[i] = random.choice(pref['mag'])
    for i in [i for i,x in enumerate(pd) if x == None]:
        pd[i] = random.choice(pref['mag'])

    #Unit exponents (and thus prefix powers)
    en =  [ i for i in SI['count'][SI['count'] > 0]] #Derived units
    en += [ i for i in remainder if i > 0]           #Base units
    ed =  [-i for i in SI['count'][SI['count'] < 0]] #Derived units
    ed += [-i for i in remainder if i < 0]           #Base units

    #Lower/raise prefixes until within user tolerance
    current_ord = sum([p*e for p,e in zip(pn,en)]) - sum([p*e for p,e in zip(pd,ed)]) - np.floor(np.log10(val))
    ratio = len(pn)/ (len(pn) + len(pd))
    numprobs = [e if p!=0 else 0 for e,p in zip(en,pn)] #Favour modifying high power values (no one wnats to see "yoctometres octed" (I kinda do though))
    numprobs = np.array(numprobs)/np.sum(np.abs(numprobs)) #  Additionally, don't select empty prefixes for change to prevent conflict with maxprefs
    denprobs = [e if p!=0 else 0 for e,p in zip(ed,pd)] #Favour modifying high power values (no one wnats to see "yoctometres octed" (I kinda do though))
    denprobs = np.array(denprobs)/np.sum(np.abs(denprobs)) #  Additionally, don't select empty prefixes for change to prevent conflict with maxprefs
    t = 0
    while current_ord > maxvalord or current_ord < minvalord:
        t+=1
        if t>100:
            print("Error, unable to meet user tolerance for output value magnitude within %i iteration attempts"%t)
            print("Keywords minvalord/maxvalord most likely define an extremely narrow range")
            raise Exception()
        if current_ord > maxvalord:
            if np.random.rand() < ratio:
                #Go down one prefix in a numerator slot
                i = np.random.choice(range(len(pn)), p=numprobs)
                newprefs = pref['mag'][pref['mag'] < pn[i]]
                if len(newprefs) == 0: continue #If there is no such prefix, reiterate
                pn[i] = min(newprefs, key=lambda x:abs(x-pn[i]))
            else:
                #Go up on prefix in a denominator slot
                i = np.random.choice(range(len(pd)), p=denprobs)
                newprefs = pref['mag'][pref['mag'] > pd[i]]
                if len(newprefs) == 0: continue
                pd[i] = min(newprefs, key=lambda x:abs(x-pd[i]))
        else:
            if np.random.rand() < ratio:
                #Go up one prefix in a numerator slot
                i = np.random.choice(range(len(pn)), p=numprobs)
                newprefs = pref['mag'][pref['mag'] > pn[i]]
                if len(newprefs) == 0: continue
                pn[i] = min(newprefs, key=lambda x:abs(x-pn[i]))
            else:
                #Go down on prefix in a denominator slot
                i = np.random.choice(range(len(pd)), p=denprobs)
                newprefs = pref['mag'][pref['mag'] < pd[i]]
                if len(newprefs) == 0: continue
                pd[i] = min(newprefs, key=lambda x:abs(x-pd[i]))
        current_ord = sum([p*e for p,e in zip(pn,en)]) - sum([p*e for p,e in zip(pd,ed)]) - np.floor(np.log10(val))

    #Calculate new numerical value of quantity
    oval = val / (10**np.floor(np.log10(val))) * 10**-current_ord

    #Convert numerical prefix orders to their corresponding strings
    pn = [pref['name'][pref['mag'] == i][0] for i in pn]
    pd = [pref['name'][pref['mag'] == i][0] for i in pd]
    if len(pd) > 0: pd[0] = ' per ' + pd[0]


    #Convert exponent values to their corresponding strings
    enames = {1:'', 2:'s-squared', 3:'s-cubed', 4:'s-quarted', 5: 's-quinted', 6: 's-sexted', 7: 's-hepted', 8: 's-octed',
              9: 'nonned????', 10: 'dude', 11: 'stop', 12: 'the', 13 : 'words', 14: ' don\'t', 15: 'even', 16: 'go', 17: 'this', 18:'high'} #If you get a key error on this you requested a BEEG obfuscation
    en = [enames[i] for i in en]
    ed = [enames[i] for i in ed]

    #Unit strings
    bnames = {0:'kilogram', 1: 'metre', 2: 'second', 3: 'ampere'}
    un =  list(SI['uname'][SI['count'] > 0])                     #Derived units
    un += [bnames[i] for i,x in enumerate(remainder) if x > 0]   #Base units
    ud =  list(SI['uname'][SI['count'] < 0])
    ud += [bnames[i] for i,x in enumerate(remainder) if x < 0]

    #Construct final value string
    ustring = ''
    for p,u,e in zip(pn, un, en):
        ustring += '-%s%s%s'%(p,u,e)
    ustring += 's' #Pluralize
    for p,u,e in zip(pd, ud, ed):
        ustring += '%s%s%s-'%(p,u,e)
    ustring = ustring[1:] #Chop off superfluous hyphens
    if len(pd) != 0:
        ustring = ustring[:-1]

    #Round oval for readability
    oval  = np.format_float_positional(oval, precision=4, unique=False, fractional=False, trim='k')
    ostring = str(oval) + ' ' + ustring

    #Cleanup plurals
    ostring = ostring.replace('hertzs', 'hertz')
    ostring = ostring.replace('henrys', 'henries')

    return ostring

In [ ]:
#Python web interactivity
from shiny import render
from shiny.express import input, ui

ui.input_text("user_input", "uit", "I just went on a 3 mile hike💪")

@render.text
def value():

    ureg = UnitRegistry()

    quantunit = parser.parse(input.user_input())[0]
    value = quantunit.value
    unit  = quantunit.unit.name

    #Convert to base SI units
    pintunit = value * ureg.parse_expression(unit)
    dims = [pintunit.dimensionality.get(name) for name in ['[mass]','[length]','[time]','[current]']]
    #Define and convert to a unit in terms of base SI units
    bu = ['kilogram','meter','second','ampere']
    baseunit = ''
    for i,u in zip(dims,bu):
        if i !=0:
            baseunit = baseunit + u + '**' + str(i) + '*'

    ureg.define('base_unit = ' + baseunit[:-1] + '= bu')
    pintunit.ito('bu')

    #Obtusify the unit
    obtuse_quant = obtusify(pintunit.magnitude, dims, derived_units, prefixes, minvalord=-8, maxvalord=8)

    obtusified_input = re.sub(quantunit.surface, obtuse_quant, input.user_input())
    return obtusified_input